### 总结
1. RunnablePassthrough +  RunnableParallel  解决检索时， 数据丢失问题
RunnablePassthrough.assign() 来有条件地添加或修改数据

# RunnablePassthrough 原理
RunnablePassthrough 的核心功能是原样传递输入数据，不做任何修改或转换。它就像一个"透明管道"，数据流经它时保持不变。

常用于处理链中，将输入数据传递，直接给下一个处理步骤。



## 1. 使用方法


In [2]:
from langchain_core.runnables import RunnablePassthrough

# 创建一个简单的 passthrough
passthrough = RunnablePassthrough()

# 使用它
result = passthrough.invoke("Hello World")
print(result)  # 输出: "Hello World"

Hello World


### 2. 在 RunnableParallel 中使用
这是最常见的使用场景，用于并行处理多个数据流：

In [ ]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

# 创建并行处理链
chain = RunnableParallel(
    # 原样传递原始输入
    original_input=RunnablePassthrough(),
    # 对输入进行某种转换
    processed_input=lambda x: x.upper()
)

result = chain.invoke("hello")
print(result)  
# 输出: {'original_input': 'hello', 'processed_input': 'HELLO'}

### 3. 在 RAG 链中的典型应用

In [ ]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

# RAG 链的构建
rag_chain = (
    RunnableParallel(
        # 检索相关文档作为上下文
        context=retriever | format_docs,
        # 保持原始问题不变
        question=RunnablePassthrough()
    )
    | prompt  # prompt 会接收 {context: "...", question: "..."}
    | llm
    | StrOutputParser()
)

# 使用时
response = rag_chain.invoke("什么是机器学习？")

### 4. 条件传递

In [ ]:
# 在传递数据的同时添加新字段
chain = (
    RunnablePassthrough.assign(
        timestamp=lambda _: datetime.now(),
        word_count=lambda x: len(x.split())
    )
)

result = chain.invoke("Hello world from LangChain")
# 结果包含原始输入 + 新增的 timestamp 和 word_count

### 5. 在复杂链中的应用

In [ ]:
# 复杂的处理链
complex_chain = (
    # 第一步：并行处理
    RunnableParallel(
        query=RunnablePassthrough(),
        embedding=lambda x: embeddings.embed_query(x)
    )
    # 第二步：基于 embedding 检索
    | RunnableParallel(
        query=lambda x: x["query"],  # 保持 query
        docs=lambda x: retriever.get_relevant_documents(x["embedding"])
    )
    # 第三步：格式化并生成回答
    | prompt
    | llm
)